# Merge 2017 information with our own

In 2017, David Lessard's team worked on Alouette and create result files which are in *L:\DATA\Alouette_I\Total\06_log\2017*. The goal of this notebook is to add those information to our image_inventory.csv and result_postprocess_raw.csv files in *L:\DATA\Alouette_I\Total\05_result*.

The first function is to add the information in *Alouette-1 scan quality qualitative observations-2017.xls* to image_inventory.csv. The function returns the new image_inventory.csv and a list of images in the 2017's file that weren't found in the image_inventory.csv (the name of the directory or subdirectory probably changed).

In [3]:
import pandas as pd
import numpy as np

def comment(file):
    
    # Import files
    image_inventory = pd.read_csv(file)
    image_inventory['Subdirectory'] = image_inventory['Subdirectory'].apply(str)
    
    comment_2017 = pd.read_excel('L:/DATA/Alouette_I/Total/06_log/2017/Alouette-1 scan quality qualitative observations-2017.xls',
                                 names = ['Directory','Subdirectory','Date','Comment','Hard_Drive'] ,nrows = 448)
    comment_2017['Subdirectory'] = comment_2017['Subdirectory'].apply(str)
    
    # Create comment column in image inventory file
    image_inventory['Comment'] = np.repeat(np.nan,len(image_inventory))
    
    # Initialize list of not found subdirectories
    notfound = pd.DataFrame(columns = ['Directory','Subdirectory','Date','Comment','Hard_Drive'])
    k = 0
    
    for i in range(len(comment_2017)):
        sub = comment_2017['Subdirectory'][i]
        idx = image_inventory[image_inventory['Subdirectory'] == sub].index
        if idx.empty :
            notfound.loc[k] = comment_2017.loc[i]
            k+=1
        else :
            image_inventory['Comment'][idx] =  comment_2017['Comment'][i]
            
    return(image_inventory,notfound)

In [4]:
file = 'L:/DATA/Alouette_I/Total/06_log/image_inventory.csv'
image_inventory, notfound = comment(file)
image_inventory.to_csv(file,index=False)

<ipython-input-3-6a57cc20d5bf>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_inventory['Comment'][idx] =  comment_2017['Comment'][i]


The second function is to add the information in *list_of_changes_to_images_1-Lessard_2017.txt*, *list_of_changes_to_images_2-Lessard_2017.txt* and *list_of_changes_to_images_3-Lessard_2017.txt* to result_postprocess_raw.csv. The function returns the new result_postprocess_raw.csv and a list of images in the 2017's file that weren't found in result_postprocess_raw.csv (the name of the directory or subdirectory probably changed). For more information on the columns of the text file, I suggest you look at them in the txt files directly.

In [5]:
def transformation(res_postprocess):

    # Initialize list of not found images
    notfoundtxt = pd.DataFrame(columns = ['Hard_Drive','Directory','Subdirectory','Image','Metadata','Rflx','Error'])
    k = 0
    
    for i in range(len(txt_2017)):
        if i%1000 == 0:
            print(str(i)+'th row on 177952...')
        sub = txt_2017['Subdirectory'][i]
        im = txt_2017['Image'][i]
        idx = res_postprocess.loc[(res_postprocess['Subdirectory'] == sub) & (res_postprocess['filename'] == im)].index
        if idx.empty :
            notfoundtxt.loc[k] = txt_2017.loc[i]
            k+=1
        else :
            res_postprocess['Change_Metadata'][idx] = txt_2017['Metadata'][i]
            res_postprocess['Change_Rflx'][idx] = txt_2017['Rflx'][i]
            res_postprocess['Change_Error'][idx] = txt_2017['Error'][i]    

    return(res_postprocess, notfoundtxt)

In [ ]:
file = 'L:/DATA/Alouette_I/BATCH_I_Run2/05_result/result_postprocess_raw.csv'
res_postprocess = pd.read_csv(file)
res_postprocess['Subdirectory'] = res_postprocess['Subdirectory'].apply(str)


txtfile = ['L:/DATA/Alouette_I/Total/06_log/2017/list_of_changes_to_images_1-Lessard_2017.txt',
           'L:/DATA/Alouette_I/Total/06_log/2017/list_of_changes_to_images_2-Lessard_2017.txt',
           'L:/DATA/Alouette_I/Total/06_log/2017/list_of_changes_to_images_3-Lessard_2017.txt']
txt1_2017 = pd.read_csv(txtfile[0],skiprows=13,sep=' ',index_col=False,
                        names=['Hard_Drive','Directory','Subdirectory','Image','Metadata','Rflx','Error'])
txt2_2017 = pd.read_csv(txtfile[1],skiprows=15,sep=' ',index_col=False,
                        names=['Hard_Drive','Directory','Subdirectory','Image','Metadata','Rflx','Error'])
txt3_2017 = pd.read_csv(txtfile[2],skiprows=13,sep=' ',index_col=False,
                        names=['Hard_Drive','Directory','Subdirectory','Image','Metadata','Rflx','Error'])
txt_2017 = pd.concat([txt1_2017,txt2_2017,txt3_2017], ignore_index=True)
txt_2017.drop_duplicates(inplace=True)
txt_2017['Subdirectory'] = txt_2017['Subdirectory'].apply(str)

#Create new columns in postprocess file
res_postprocess['Change_Metadata'] = np.repeat(np.nan,len(res_postprocess))
res_postprocess['Change_Rflx'] = np.repeat(np.nan,len(res_postprocess))
res_postprocess['Change_Error'] = np.repeat(np.nan,len(res_postprocess))

res_postprocess, notfoundtxt = transformation(res_postprocess)
res_postprocess.to_csv(file,index=False)